In [2]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import time

import networkx as nx
from community import community_louvain

In [3]:
df = pd.read_csv('data/corpus.csv').drop(columns = 'Unnamed: 0')

In [4]:
len(df)

1032133

In [6]:
df.head(1)

,index,user_identifier,description,date,longitude,latitude,external_upload_mean,camera,license_name,flickr_url,image_direct_url,user_tags,autotags
0,10000979206,66172503@N00,NaN,2013-09-30 00:26:09.0,14.499968,46.053249,"foursquare%3Avenue%3D4e2b1e5a315185636a08f0dd,...",NaN,Attribution License,http://www.flickr.com/photos/66172503@N00/1000...,http://farm4.staticflickr.com/3702/10000979206...,"instagram+app,iphoneography,nashville,square,s...","abstract:0.8090,architecture:0.9030,bright:0.6..."


In [10]:
len(df['date'].dropna())

1032100

In [4]:
proba = []

for ind, item in df['autotags'].iteritems():
    
    tags = re.findall('[A-z\s\-\']+', str(item))
    values = re.findall('[0-9\.]+', str(item))
    values = np.asarray(values).astype('float')
    
    for i in range(len(tags)):
        if tags[i] == 'minimalism':
            proba.append(values[i])

In [5]:
len(proba)

1027777

In [6]:
df['minimalism_proba'] = np.nan

In [7]:
df_ = df[~df['autotags'].isna()]

In [8]:
df['minimalism_proba'][df_.index] = proba

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df.to_csv('data/corpus_plus.csv')

In [10]:
df = df[~df['autotags'].isna()]
df = df[df['minimalism_proba'] > 0.8]
df.head()

,index,user_identifier,description,date,longitude,latitude,external_upload_mean,camera,license_name,flickr_url,image_direct_url,user_tags,autotags,minimalism_proba
11,10047554334,27443270@N00,SONY+DSC,2013-09-19 17:58:25.0,NaN,NaN,NaN,SONY+DSLR-A700,Attribution-ShareAlike License,http://www.flickr.com/photos/27443270@N00/1004...,http://farm4.staticflickr.com/3768/10047554334...,9-23-13+an+adirondack+weekend+a700,"aircraft:0.9990,balloon:0.9990,blue:0.7200,bri...",0.885
15,10057499003,8739016@N07,This+was+my+first+Striper+in+6+years+or+so.,2013-09-22 22:57:35.0,NaN,NaN,NaN,OLYMPUS+IMAGING+CORP.+TG-820,Attribution License,http://www.flickr.com/photos/8739016@N07/10057...,http://farm6.staticflickr.com/5447/10057499003...,"fishing,striperheads,travel","abstract:0.6600,animal:0.6600,jewelry:0.6290,m...",0.905
22,10093026355,45682148@N05,NaN,2013-10-05 09:09:23.0,NaN,NaN,uploaded%3Aby%3Dinstagram,NaN,Attribution-ShareAlike License,http://www.flickr.com/photos/45682148@N05/1009...,http://farm6.staticflickr.com/5474/10093026355...,"instagram+app,iphoneography,square,square+form...","abstract:0.7470,art:0.6900,handmade:0.7390,min...",0.933
23,10093247345,67093783@N00,NaN,2013-10-04 17:11:28.0,NaN,NaN,NaN,Canon+PowerShot+SX260+HS,Attribution-NonCommercial-ShareAlike License,http://www.flickr.com/photos/67093783@N00/1009...,http://farm8.staticflickr.com/7432/10093247345...,NaN,"aerial:0.9030,aircraft:0.9930,airplane wing:0....",0.849
32,10119178236,43563891@N00,Section+of+the+bathroom+and+skylight+in+the+US...,2013-10-05 17:49:46.0,-117.730951,33.674211,NaN,HTC6435LVW,Attribution-ShareAlike License,http://www.flickr.com/photos/43563891@N00/1011...,http://farm4.staticflickr.com/3685/10119178236...,NaN,"floor:0.5040,indoor:0.6660,lamp:0.6780,light:0...",0.816


In [11]:
len(df)

198655

In [12]:
autotags_list = []

for ind, item in df['autotags'].iteritems():
    tags = re.split('[,:0-9]', str(item))
    for tag in tags:
        if len(tag) > 1:
            autotags_list.append(tag)

In [13]:
autotags_list = pd.Series(autotags_list).value_counts()

In [14]:
print(len(autotags_list))

1470


In [15]:
autotags_list.to_json('json/autotags_list')

In [16]:
autotags_list[:20]

minimalism          198655
abstract            143965
texture             122181
pattern             115685
nature              110670
outdoor             101727
symmetry             86604
serene               85845
text                 84524
diagonal             82948
geometric            69362
lines                68278
art                  64865
quirky               61447
black                59364
surreal              56139
architecture         55973
depth of field       54046
rain                 48874
black background     48858
dtype: int64

In [17]:
array = pd.DataFrame(np.zeros(len(autotags_list)), autotags_list.index)

In [18]:
for tag in autotags_list.index:
    array[tag] = np.zeros(len(autotags_list))

array = array.drop(columns = 0)

In [19]:
for ind, item in df['autotags'].iteritems():
    
    tags = re.findall('[A-z\s\-\']+', str(item))
    values = re.findall('[0-9\.]+', str(item))
    values = np.asarray(values).astype('float')
    
    for one in range(len(tags)):
        for two in range(len(tags)):
            try:
                if (values[one] >= values[two]):
                    array[tags[one]][tags[two]] += values[one]*values[two]
                else:
                    array[tags[two]][tags[one]] += values[one]*values[two]
            except:
                pass

In [20]:
array.to_csv('tags_colocation_matrix.csv')

In [21]:
array

,minimalism,abstract,texture,pattern,nature,outdoor,symmetry,serene,text,diagonal,...,villa,lioness,armored personnel carrier,black swan,caribou,meadow cranesbill,hospital,guanaco,cattleya,curry
minimalism,153649.594470,116275.439266,82234.607562,36958.820215,55966.711664,109604.931111,7715.789447,17075.458492,38623.403000,25966.238151,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
abstract,96514.711010,103070.943041,62406.265558,24592.992470,38021.030467,47744.764270,10146.924760,17552.947999,28187.273713,20142.873787,...,0.000000,0.631446,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
texture,93307.461586,98132.204058,82981.541443,36762.393996,37220.485676,41337.655072,14359.480856,19365.152800,28452.781620,27645.386616,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.7332
pattern,117939.507973,116595.915650,93375.639570,67814.500187,45765.621362,53198.180228,18869.051792,25476.045906,41411.798878,36609.031506,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
nature,93171.857124,60327.762929,57653.398038,34210.779376,67509.422160,76870.100474,16127.097515,26767.899183,25084.742601,22516.499797,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.7722
outdoor,49460.284585,42194.013362,37478.766808,16408.907852,25530.175718,82144.234713,4949.669923,7912.562419,15086.522309,12317.205457,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
symmetry,99850.815115,82579.399064,63182.338808,61715.783188,32170.095395,42334.785333,43099.054481,23944.649063,38786.339683,34561.557242,...,0.000000,0.000000,0.00000,0.000000,0.819764,0.997272,0.000000,0.000000,0.000000,0.0000
serene,89013.436084,53502.716289,50786.806740,33387.651576,53460.994319,62122.892021,16789.810201,43562.713676,23218.630865,19815.578675,...,0.000000,0.000000,0.70082,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
text,76690.060776,67886.006567,54070.487098,31693.070094,20809.408955,31390.576293,14023.593175,13399.060795,51727.134704,23180.358707,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000
diagonal,85594.529215,83173.348211,53131.462184,46963.275200,24332.288617,38605.257521,20688.193558,15049.355683,26472.439625,48531.200282,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000


In [22]:
G = nx.Graph()
G.add_nodes_from(array.columns)

In [23]:
list_edges = []

for ind, row in array.iterrows():
    for column in array.columns:
        if row[column] > 0.:
            list_edges.append((ind, column, row[column]/2.))

In [24]:
G.add_weighted_edges_from(list_edges)

In [25]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1470
Number of edges: 277687
Average degree: 377.8054


In [26]:
partition = community_louvain.best_partition(G)

In [27]:
partition = pd.Series(partition)

In [37]:
partition

minimalism                   0
abstract                     1
texture                      1
pattern                      1
nature                       2
outdoor                      2
symmetry                     1
serene                       2
text                         0
diagonal                     1
geometric                    1
lines                        1
art                          0
quirky                       0
black                        3
surreal                      0
architecture                 1
depth of field               4
rain                         1
black background             3
bright                       4
organic pattern              4
urban decay                  1
indoor                       0
pastel                       0
animal                       4
water                        2
writing                      0
landscape                    2
white                        0
                            ..
mammal                       4
stagecoa

In [28]:
partition.value_counts()

1    479
0    315
4    304
2    293
3     79
dtype: int64

In [33]:
partition[partition == 0].index[:25] # indoor, food, people => USA ?

Index(['minimalism', 'text', 'art', 'quirky', 'surreal', 'indoor', 'pastel',
       'writing', 'white', 'food', 'white background', 'romantic', 'hands',
       'beverage', 'handmade', 'alcohol', 'wine', 'jewelry', 'people', 'head',
       'drawing', 'painting', 'coffee', 'watercolor', 'cartoon'],
      dtype='object')

In [34]:
partition[partition == 1].index[:25] # urban, geometry => Asie / Japon ?

Index(['abstract', 'texture', 'pattern', 'symmetry', 'diagonal', 'geometric',
       'lines', 'architecture', 'rain', 'urban decay', 'curve', 'sport',
       'grid', 'building', 'alley', 'photo border', 'blackandwhite', 'sign',
       'street sign', 'city', 'cityscape', 'track', 'gray', 'passageway',
       'electronics'],
      dtype='object')

In [35]:
partition[partition == 2].index[:25] # landscape, sky, water

Index(['nature', 'outdoor', 'serene', 'water', 'landscape', 'vehicle', 'blue',
       'ethereal', 'sea', 'fog', 'dusk', 'aircraft', 'windmill', 'sunlight',
       'sunset', 'sky', 'field', 'sun', 'skyline', 'airplane', 'sand',
       'meadow', 'snow', 'rainbow', 'lake'],
      dtype='object')

In [36]:
partition[partition == 3].index[:25] # light effect, circular => Scandinavie ?

Index(['black', 'black background', 'monochrome', 'night', 'lunar', 'moon',
       'neon light', 'light painting', 'fractal', 'round', 'circle', 'light',
       'full moon', 'lamp', 'lampshade', 'fire', 'flame', 'bubble',
       'room light', 'ancient', 'satellite', 'photomicrograph', 'aurora',
       'jellyfish', 'fireworks'],
      dtype='object')

In [46]:
partition[partition == 4].index[:25] # plants

Index(['depth of field', 'bright', 'organic pattern', 'animal', 'plant',
       'macro', 'bokeh', 'foliage', 'insect', 'autumn', 'green', 'blur',
       'leaf', 'fly', 'yellow', 'red', 'underwater', 'tree', 'grass',
       'blossom', 'flower', 'purple', 'dewdrop', 'forest', 'dark red'],
      dtype='object')

In [40]:
df_tot = pd.read_csv('data/corpus.csv').drop(columns = 'Unnamed: 0')
df_tot = df_tot[~df_tot['autotags'].isna()]

In [41]:
new_autotags = []
for ind, item in df_tot['autotags'].iteritems():
    row = []
    tags = re.findall('[A-z\s\-\']+', str(item))
    values = re.findall('[0-9\.]+', str(item))
    values = np.asarray(values).astype('float')
    
    for i in range(len(tags)):
        row.append((tags[i], values[i]))
    new_autotags.append(row)
    

In [44]:
df_tot['json_autotags'] = new_autotags

In [53]:
louvain_theme_cluster = []

for ind, item in df_tot['json_autotags'].iteritems():
    partition_prob = np.zeros(5)
    for i in range(len(item)):
        try:
            cluster = partition[item[i][0]]
            partition_prob[cluster] += item[i][1]
        except:
            pass
    if np.sum(partition_prob) == 0.:
        louvain_theme_cluster.append(np.nan)
    else:
        louvain_theme_cluster.append(int(np.argmax(partition_prob)))

In [54]:
df_tot['tag_cluster'] = louvain_theme_cluster

In [59]:
df_tot = df_tot.drop(columns = ['description', 'flickr_url', 'image_direct_url',
                               'user_tags', 'autotags'])

In [105]:
tags = ['instagram', 'flickrandroidapp', 'flickriosapp', 'taxonomy', 'camera',
       'lastfm', 'astrometrydotnet', 'foursquare', 'geocode', 'geo', 'dopplr', 'inaturalist', 'http']

external_upload = []
for ind, item in df_tot['external_upload_mean'].iteritems():
    text = str(item)
    new_item = np.nan
    if ~(text == 'nan'):
        for tag in tags:
            found = re.findall(tag, text)
            if len(found) > 0:
                new_item = tag
                break
    external_upload.append(new_item)

In [108]:
df_tot['external_upload'] = external_upload

In [109]:
df_tot = df_tot.drop(columns = 'external_upload_mean')

In [110]:
df_tot.head(1)

,index,user_identifier,date,longitude,latitude,camera,license_name,json_autotags,tag_cluster,external_upload
0,10000979206,66172503@N00,2013-09-30 00:26:09.0,14.499968,46.053249,NaN,Attribution License,"[(abstract, 0.809), (architecture, 0.903), (br...",1,instagram


In [ ]:
dummies_license = pd.get_dummies(df['license_name'])
dummies_upload = pd.get_dummies(df['external_upload'])